In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

from network import Network
from utils import device, get_all_preds, get_num_correct

In [2]:
model = Network().to(device)
model.load_state_dict(torch.load('models/model-run(lr=0.003, batch_size=256).ckpt', map_location=device))

<All keys matched successfully>

In [13]:
model

Network(
  (layer1): Sequential(
    (0): Conv2d(1, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Dropout(p=0.2, inplace=False)
    (3): ReLU()
  )
  (fc2): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2):

In [3]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

train_set = torchvision.datasets.FashionMNIST(
    root='./data/',
    train=True,
    download=True,
    transform=transform
)
test_set = torchvision.datasets.FashionMNIST(
    root='./data/',
    train=False,
    download=True,
    transform=transform
)

In [4]:
model.eval()
with torch.no_grad():
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=6000)
    train_preds = get_all_preds(model, train_loader)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=1000)
    test_preds = get_all_preds(model, test_loader)
    print(train_preds.shape)
    print(test_preds.shape)

torch.Size([60000, 10])
torch.Size([10000, 10])


In [7]:
train_correct = get_num_correct(train_preds, train_set.targets)
test_correct = get_num_correct(test_preds, test_set.targets)

print('Train Correct: {:5}\tTrain Accuracy: {:5.2f}%'.format(train_correct, 100*train_correct/len(train_set)))
print('Test Correct: {:6}\tTest Accuracy: {:6.2f}%'.format(test_correct, 100*test_correct/len(test_set)))

Train Correct: 59018	Train Accuracy: 98.36%
Test Correct:   9108	Test Accuracy:  91.08%


In [8]:
train_stacked = torch.stack(
    (train_set.targets, train_preds.argmax(dim=1)),
    dim=1
)
test_stacked = torch.stack(
    (test_set.targets, test_preds.argmax(dim=1)),
    dim=1
)
train_confmat = torch.zeros(10, 10, dtype=torch.int16)
test_confmat = torch.zeros(10, 10, dtype=torch.int16)

In [9]:
print(train_set.classes)
for row in train_stacked:
    cl, pl = row.tolist()
    train_confmat[cl, pl] += 1

train_confmat

['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


tensor([[5764,    0,    7,   33,    1,    1,  194,    0,    0,    0],
        [   1, 5989,    0,   10,    0,    0,    0,    0,    0,    0],
        [  28,    0, 5866,    4,   50,    0,   52,    0,    0,    0],
        [   8,    0,    4, 5966,   11,    0,   11,    0,    0,    0],
        [   3,    0,  127,   32, 5803,    0,   35,    0,    0,    0],
        [   0,    0,    0,    0,    0, 5994,    0,    5,    0,    1],
        [ 110,    0,   93,   18,   57,    0, 5721,    0,    1,    0],
        [   0,    0,    0,    0,    0,   11,    0, 5985,    0,    4],
        [   1,    0,    1,    0,    1,    0,    0,    0, 5997,    0],
        [   0,    0,    0,    0,    0,    5,    0,   62,    0, 5933]],
       dtype=torch.int16)

In [10]:
print(test_set.classes)
for row in test_stacked:
    cl, pl = row.tolist()
    test_confmat[cl, pl] += 1

test_confmat

['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


tensor([[841,   0,  12,  15,   3,   0, 124,   0,   5,   0],
        [  1, 978,   0,  14,   1,   0,   3,   0,   3,   0],
        [ 21,   1, 863,   9,  47,   0,  59,   0,   0,   0],
        [ 13,   2,   8, 928,  22,   0,  27,   0,   0,   0],
        [  2,   0,  65,  26, 845,   0,  61,   0,   1,   0],
        [  0,   0,   0,   0,   0, 987,   0,  10,   0,   3],
        [ 90,   0,  65,  26,  67,   0, 747,   0,   5,   0],
        [  0,   0,   0,   0,   0,   6,   0, 982,   1,  11],
        [  2,   1,   2,   3,   3,   2,   3,   2, 982,   0],
        [  1,   0,   0,   0,   1,   4,   0,  39,   0, 955]], dtype=torch.int16)

In [11]:
for i in range(10):
    print('Train accuracy of {:12s}:\t{:.2f}% ({}/{})'.format(
        train_set.classes[i],
        train_confmat[i, i]/60,
        train_confmat[i, i],
        6000))

Train accuracy of T-shirt/top :	96.07% (5764/6000)
Train accuracy of Trouser     :	99.82% (5989/6000)
Train accuracy of Pullover    :	97.77% (5866/6000)
Train accuracy of Dress       :	99.43% (5966/6000)
Train accuracy of Coat        :	96.72% (5803/6000)
Train accuracy of Sandal      :	99.90% (5994/6000)
Train accuracy of Shirt       :	95.35% (5721/6000)
Train accuracy of Sneaker     :	99.75% (5985/6000)
Train accuracy of Bag         :	99.95% (5997/6000)
Train accuracy of Ankle boot  :	98.88% (5933/6000)


In [12]:
for i in range(10):
    print('Test accuracy of {:12s}:\t{:.2f}% ({}/{})'.format(
        train_set.classes[i],
        test_confmat[i, i]/10,
        test_confmat[i, i],
        1000))

Test accuracy of T-shirt/top :	84.10% (841/1000)
Test accuracy of Trouser     :	97.80% (978/1000)
Test accuracy of Pullover    :	86.30% (863/1000)
Test accuracy of Dress       :	92.80% (928/1000)
Test accuracy of Coat        :	84.50% (845/1000)
Test accuracy of Sandal      :	98.70% (987/1000)
Test accuracy of Shirt       :	74.70% (747/1000)
Test accuracy of Sneaker     :	98.20% (982/1000)
Test accuracy of Bag         :	98.20% (982/1000)
Test accuracy of Ankle boot  :	95.50% (955/1000)
